In [26]:
# streamlit run streamlit_site/streamlit_app.py
# Imports
import os
import re
import pandas as pd
from sqlalchemy import (
    select, create_engine, text, Table, Column, Integer, String, MetaData, ForeignKey, LargeBinary)
from sqlalchemy.orm import sessionmaker
import pdfplumber
import fitz  # PyMuPDF
# For Image Display within the df
from IPython.display import Image
from PIL import Image, ImageOps
from io import BytesIO
from datetime import datetime
from collections import defaultdict

In [2]:
# Intializing database connection
# Replace values with your actual database info
username = "tpmpams_user"
password = "X5Lx2fWLXQ18cxaEngOODl3gXtMq7H8f"
host = "dpg-d0r91k2dbo4c73a4kip0-a.singapore-postgres.render.com"
port = "5432"
database = "tpmpams"

# SQLAlchemy connection URL
DATABASE_URL = f"postgresql://{username}:{password}@{host}:{port}/{database}"

# Create engine
engine = create_engine(DATABASE_URL)

In [171]:
# Creating the 2 tables
metadata = MetaData()

pdf_log = Table(
    "pdf_log", metadata,
    Column("pdf_id", String, primary_key=True),
    Column("staff", String, nullable=False),
    Column("timestamp", String, nullable=False),
    Column("is_current", Integer, nullable=False),
)

master_parts_list = Table(
    "master_parts_list", metadata,
    Column("mpl_id", Integer, primary_key=True, autoincrement=True),
    Column("pdf_id", String, ForeignKey("pdf_log.pdf_id", ondelete="CASCADE")),
    Column("year", Integer, nullable=False),
    Column("brand", String, nullable=False),
    Column("model", String, nullable=False),
    Column("batch_id", String, nullable=False),
    Column("section", Integer, nullable=False),
    Column("component_name", String, nullable=False),
    Column("ref_no", Integer, nullable=False),
    Column("part_no", String, nullable=False),
    Column("description", String, nullable=False),
    Column("add_info", String),
    Column("image_id", String, ForeignKey("parts_images.image_id", ondelete="SET NULL"))
)

parts_images = Table(
    "parts_images", metadata,
    Column("image_id", String, primary_key=True),
    Column("pdf_id", String, ForeignKey("pdf_log.pdf_id", ondelete="CASCADE")),
    Column("image", LargeBinary, nullable=False),
)

# Create the tables
metadata.create_all(engine)

In [ ]:
# Old code (For Reference)
# ---------- HELPERS ----------
def extract_pdf_id(pdf_path):
    # To-Do: modify pdf_id extraction to suit both pdf format
    base_filename = os.path.basename(pdf_path).split('.')[0]
    match = re.match(r"([A-Za-z0-9 ]+)", base_filename)
    if match:
        return match.group(1).replace(" ", "")  # Remove all spaces
    return None

def extract_year(pdf_path):
    year_match = re.search(r"'(\d{2})", pdf_path)
    return f"20{year_match.group(1)}" if year_match else None

def extract_model(pdf_path):
    base_filename = os.path.basename(pdf_path)
    match = re.search(r"\((.*?)\)", base_filename)
    if match:
        return match.group(1)  # e.g., "B65P, B65R, B65S"
    return None

# ---------- IMAGE EXTRACTION ----------
def normalize_image_background(image_bytes):
    img = Image.open(BytesIO(image_bytes)).convert("L")  # Grayscale
    mean_brightness = sum(img.getdata()) / (img.width * img.height)

    if mean_brightness < 128:  # Invert if it's a dark background
        img = ImageOps.invert(img)

    img = img.convert("RGB")  # Convert back to RGB

    output = BytesIO()
    img.save(output, format="PNG")
    return output.getvalue()

def get_existing_fig_combos(engine, pdf_id):
    with engine.connect() as conn:
        result = conn.execute(
            text("SELECT section FROM parts_images WHERE pdf_id = :pdf_id"),
            {"pdf_id": pdf_id}
        )
        return set(str(row[0]) for row in result.fetchall())  # Cast to str for consistent comparison

def extract_images_with_fig_labels(pdf_path, pdf_id, engine):
    doc = fitz.open(pdf_path)
    data = []

    # Step 1: Get existing (pdf_id, fig_no) combos from DB
    existing_figs = get_existing_fig_combos(engine, pdf_id)

    seen_figs = set()  # Track unique figs within the PDF

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text()

        matches = re.findall(r"FIG\.\s*([\w-]+)", text)
        if not matches:
            continue

        section = matches[0]

        if section in seen_figs or section in existing_figs:
            continue  # Skip if already handled or exists in DB

        image_list = page.get_images(full=True)
        if not image_list:
            continue

        xref = image_list[0][0]
        base_image = doc.extract_image(xref)
        image = normalize_image_background(base_image["image"])

        image_id = "_".join([pdf_id, section])

        data.append({
            "image_id" : image_id,
            "pdf_id": pdf_id,
            "section": section,
            "image": image
        })
        seen_figs.add(section)

    return pd.DataFrame(data)

# ---------- TEXT EXTRACTION ----------
def extract_text_from_pdf(pdf_path):
    all_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            text = page.extract_text() or ""
            lines = text.split('\n')
            first_line = lines[0].strip() if lines else ""
            if not any("FIG." in line for line in lines):
                continue
            if "NUMERICAL INDEX" in first_line:
                break
            all_text += f"\n--- Page {page_num + 1} ---\n{text}\n"
    return all_text

def yamaha_process_data(text, pdf_id, year, model, num_model):
    rows = []
    lines = text.strip().split('\n')
    section = c_name = prev_fig_no = prev_c_name = prev_ref_no = ""
    collect_data = False

    for line in lines:
        line = line.strip()
        if not line: continue
        if line.startswith('FIG.'):
            tokens = line.split()
            if len(tokens) >= 3:
                section = tokens[1]
                c_name = " ".join(tokens[2:])
                prev_fig_no, prev_c_name = section, c_name
                collect_data = True
            continue
        if not collect_data: continue
        if not section:
            section, c_name = prev_fig_no, prev_c_name

        parts = line.split()
        is_valid_data_line = (
            len(parts) >= 2 and 
            (re.match(r'\w+[-–]\w+', parts[0]) or parts[0].isdigit())
        )
        if not is_valid_data_line:
            continue

        if parts[0].isdigit():
            ref_no = parts[0]
            part_no = parts[1]
            rest = parts[2:]
            prev_ref_no = ref_no
        else:
            ref_no = prev_ref_no
            part_no = parts[0]
            rest = parts[1:]

        rest = " ".join(rest).split()
        description = remarks = ""
        numbers = []
        found_numbers = False
        for item in rest:
            if item.isdigit():
                numbers.append(item)
                found_numbers = True
                continue
            if not found_numbers:
                description += item + " "
            else:
                remarks += item + " "
        if len(numbers) > num_model:
            description += numbers[0]

        image_id = "_".join([pdf_id, section])

        rows.append([pdf_id, year, "Yamaha", model, section, c_name, ref_no, part_no, description.strip(), remarks.strip(), image_id])

    return pd.DataFrame(rows, columns=[
        'pdf_id', 'year', 'brand', 'model', 'section', 'component_name',
        'ref_no', 'part_no', 'description', 'remarks', 'image_id'
    ])

# ---------- MAIN PROCESS ----------
def yamaha_data_extraction(pdf_path):

    pdf_id = extract_pdf_id(pdf_path)
    year = extract_year(pdf_path)
    model = extract_model(pdf_path)

    SessionLocal = sessionmaker(bind=engine)
    session = SessionLocal()
    try:
        # -------- pdf_log table --------
        existing_log = session.execute(
            select(1).select_from(pdf_log).where(pdf_log.c.pdf_id == pdf_id)
        ).first()

        if not existing_log:
            session.execute(
                pdf_log.insert().values(
                    pdf_id=pdf_id,
                    timestamp=datetime.now().isoformat()
                )
            )
            session.commit()
            print(f"[INFO] Logged PDF '{pdf_id}' in pdf_log.")
        else:
            print(f"[INFO] PDF '{pdf_id}' already logged.")

        # -------- parts_images table --------
        df_images = extract_images_with_fig_labels(pdf_path, pdf_id, engine)
        image_message = f"[INFO] Inserted {len(df_images)} new images for '{pdf_id}'."
        if not df_images.empty:
            df_images.to_sql("parts_images", engine, if_exists="append", index=False, method="multi")
            print(image_message)
        else:
            print(image_message + f" All images for '{pdf_id}' already exist.")

        existing = session.execute(
            select(1).select_from(master_parts_list).where(master_parts_list.c.pdf_id == pdf_id)
        ).first()

        if existing:
            print(f"[INFO] Master Parts data for '{pdf_id}' already exists.")
            return

    finally:
        session.close()
            
    # -------- master_parts_list table --------
    all_text = extract_text_from_pdf(pdf_path)
    df_parts = yamaha_process_data(all_text, pdf_id, year, model, num_model=3)

    if not df_parts.empty:
        #print(df_parts.to_string(index=False))
        df_parts.to_sql("master_parts_list", engine, if_exists="append", index=False, method="multi")
        print(f"[INFO] Inserted parts data for '{pdf_id}'.")
    else:
        print(f"[INFO] Error, no parts data extracted for '{pdf_id}'.")

In [ ]:
# Old code (For Refernece)
# Format 1: Yamaha, Important images from page 6-60
pdf_1 = "Manuals/AEROX 155 '19 (B65P, B65R, B65S).pdf"
pdf_2= "Manuals/FJR1300A '15 (1MCH, 1MCG).PDF"
# Format 2: Honda
pdf_3 = "Manuals/CRF1000 A_PC_13MJPG02_(G.H).pdf"
pdf_4 = "Manuals/NC750XAP_13MKWM02_PC_2022_2023.pdf"

pdf_path = pdf_1

brand = "Yamaha"
supported_brands = ['Yamaha', 'Honda']

if brand in supported_brands:
    if brand == "Yamaha":
        yamaha_data_extraction(pdf_path)
    elif brand == "Honda":
        print("Hi")
else:
    print (f'"{brand}" not supported \nAvailable Brands: {supported_brands}')

In [ ]:
# Testing
# Format 1: Yamaha
pdf_1 = "Manuals/AEROX 155 '19 (B65P, B65R, B65S).pdf"
pdf_2= "Manuals/FJR1300A '15 (1MCH, 1MCG).PDF"
# Format 2: Honda
pdf_3 = "Manuals/CRF1000 A_PC_13MJPG02_(G.H).pdf"
pdf_4 = "Manuals/NC750XAP_13MKWM02_PC_2022_2023.pdf"

def model_extraction(pdf_path):
    base_filename = os.path.basename(pdf_path).split('.')[0]

    # Extract model: start of filename, letters/numbers/spaces until a special character (', _)
    match = re.match(r"([A-Za-z0-9 ]+)", base_filename)
    if match:
        return match.group(1).replace(" ", "")  # Removes any spaces
    
    return None

def batch_id_extraction(pdf_path, brand):
    base_filename = os.path.basename(pdf_path).split('.')[0]

    if brand == "Yamaha":
        # Extract model codes inside parentheses
        match = re.search(r"\((.*?)\)", base_filename)
        if match:
            parts = match.group(1).split(",")
            clean_parts = [part.strip() for part in parts]
            return "_".join(clean_parts)
    
    elif brand == "Honda":
        # Look for uppercase/digit code between underscores (6–10 characters)
        match = re.search(r"_([A-Z0-9]{6,10})_", base_filename)
        if match:
            return match.group(1)

    return None

def year_extraction(pdf_path, brand):
    base_filename = os.path.basename(pdf_path)

    if brand == "Yamaha":
        year_match = re.search(r"'(\d{2})", base_filename)
        return f"20{year_match.group(1)}" if year_match else None

    # elif brand == "Honda":
    #     # Look for a full year range like 2022_2023
    #     match = re.search(r"(20\d{2}_20\d{2})", base_filename)
    #     return match.group(1) if match else None

    return None

print(year_extraction (pdf_1, "Yamaha"))
print(year_extraction (pdf_2, "Yamaha"))
print(year_extraction (pdf_3, "Honda"))
print(year_extraction (pdf_4, "Honda"))

2019
2015
None
2022_2023


In [ ]:
class PDFProcessor:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path
        self.base_filename = os.path.basename(pdf_path).split('.')[0]

    def model_extraction(pdf_path):
        base_filename = os.path.basename(pdf_path).split('.')[0]

        # Extract model: start of filename, letters/numbers/spaces until a special character (', _)
        match = re.match(r"([A-Za-z0-9 ]+)", base_filename)
        if match:
            return match.group(1).replace(" ", "")  # Removes any spaces
        
        return None

    def batch_id_extraction(pdf_path, brand):
        base_filename = os.path.basename(pdf_path).split('.')[0]

        if brand == "Yamaha":
            # Extract model codes inside parentheses
            match = re.search(r"\((.*?)\)", base_filename)
            if match:
                parts = match.group(1).split(",")
                clean_parts = [part.strip() for part in parts]
                return "_".join(clean_parts)
        
        elif brand == "Honda":
            # Look for uppercase/digit code between underscores (6–10 characters)
            match = re.search(r"_([A-Z0-9]{6,10})_", base_filename)
            if match:
                return match.group(1)

        return None

    def year_extraction(pdf_path, brand):
        base_filename = os.path.basename(pdf_path)

        if brand == "Yamaha":
            year_match = re.search(r"'(\d{2})", base_filename)
            return f"20{year_match.group(1)}" if year_match else None

        # elif brand == "Honda":
        #     # Look for a full year range like 2022_2023
        #     match = re.search(r"(20\d{2}_20\d{2})", base_filename)
        #     return match.group(1) if match else None

        return None

class YamahaPDFProcessor(PDFProcessor):
    def extract_model(self):
        match = re.match(r"([A-Za-z0-9 ]+)", self.base_filename)
        return match.group(1).replace(" ", "") if match else None

    def extract_batch_id(self):
        match = re.search(r"\((.*?)\)", self.base_filename)
        if match:
            return "_".join(part.strip() for part in match.group(1).split(","))
        return None

    def extract_year(self):
        match = re.search(r"'(\d{2})", self.pdf_path)
        return f"20{match.group(1)}" if match else None

    def extract_raw_text(self):
        return extract_yamaha_raw_text(self.pdf_path)

class HondaPDFProcessor(PDFProcessor):
    def extract_model(self):
        match = re.match(r"([A-Za-z0-9 ]+)", self.base_filename)
        return match.group(1).replace(" ", "") if match else None

    def extract_batch_id(self):
        match = re.search(r"_([A-Z0-9]{6,10})_", self.base_filename)
        return match.group(1) if match else None

    def extract_year(self):
        match = re.search(r"(20\d{2}_20\d{2})", self.base_filename)
        return match.group(1) if match else None

    def extract_raw_text(self):
        print("[INFO] Honda raw text extraction not implemented.")
        return []

In [ ]:
# Yamaha + Honda
def extract_model(pdf_path):
    base_filename = os.path.basename(pdf_path).split('.')[0]

    # Extract model: start of filename, letters/numbers/spaces until a special character (', _)
    match = re.match(r"([A-Za-z0-9 ]+)", base_filename)
    if match:
        return match.group(1).replace(" ", "")  # Removes any spaces
    
    return None
# Yamaha + Honda
def extract_batch_id(pdf_path, brand):
    base_filename = os.path.basename(pdf_path).split('.')[0]

    if brand == "Yamaha":
        # Extract model codes inside parentheses
        match = re.search(r"\((.*?)\)", base_filename)
        if match:
            parts = match.group(1).split(",")
            clean_parts = [part.strip() for part in parts]
            return "_".join(clean_parts)
    
    elif brand == "Honda":
        # Look for uppercase/digit code between underscores (6–10 characters)
        match = re.search(r"_([A-Z0-9]{6,10})_", base_filename)
        if match:
            return match.group(1)

    return None
# Yamaha
def extract_year(pdf_path, brand):
    base_filename = os.path.basename(pdf_path)

    if brand == "Yamaha":
        year_match = re.search(r"'(\d{2})", base_filename)
        return f"20{year_match.group(1)}" if year_match else None

    # elif brand == "Honda":
    #     # Look for a full year range like 2022_2023
    #     match = re.search(r"(20\d{2}_20\d{2})", base_filename)
    #     return match.group(1) if match else None

    return None

def reconstruct_lines_from_chars(chars, y_tolerance=2.5):
    lines = defaultdict(list)

    for c in chars:
        # Use midpoint instead of top
        y_center = c["top"] + (c["height"] / 2)
        y_bucket = round(y_center / y_tolerance)
        lines[y_bucket].append(c)

    line_texts = []
    for y in sorted(lines.keys()):
        chars_in_line = sorted(lines[y], key=lambda c: c["x0"])
        line = ""
        prev_x = None

        for char in chars_in_line:
            x = char["x0"]
            text = char["text"]

            if prev_x is not None:
                gap = x - prev_x
                if gap > 1.5:
                    line += " " * int(gap / 2.5)

            line += text
            prev_x = char["x1"]

        line_texts.append((y, line.rstrip()))

    return line_texts

def extract_raw_text(pdf_path):
    output_lines = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            chars = page.chars
            raw_lines = reconstruct_lines_from_chars(chars, y_tolerance=5.5)

            # Skip non-parts pages
            if not raw_lines or not raw_lines[0][1].strip().startswith("FIG."):
                continue

            for _, line in raw_lines:
                stripped_line = line.strip()

                # ✅ Skip lines that are just a number (likely page numbers)
                if re.fullmatch(r"\d+", stripped_line):
                    continue

                if stripped_line:
                    output_lines.append(stripped_line)

    return output_lines

def structure_raw_text(raw_lines):
    structured_output = []
    skip_indices = set()

    for i in range(len(raw_lines)):
        if i in skip_indices:
            continue

        line = raw_lines[i].strip()
        parts = re.split(r"\s{2,}", line)

        # --- Normalize FIG. rows to always be ['FIG.', 'number', 'description']
        if parts and isinstance(parts[0], str) and parts[0].startswith("FIG."):
            if re.match(r"^FIG\.\s*\d+$", parts[0]):
                match = re.match(r"^(FIG\.)\s*(\d+)$", parts[0])
                if match:
                    parts = [match.group(1), match.group(2)] + parts[1:]

            elif re.match(r"^FIG\.\d+$", parts[0]):
                match = re.match(r"^(FIG\.)(\d+)$", parts[0])
                if match:
                    parts = [match.group(1), match.group(2)] + parts[1:]

        # --- Skip rows that are just floating descriptions
        if len(parts) == 1 and re.match(r"^[A-Z ,\-0-9]+$", parts[0]):
            continue

        # --- Heuristic: Missing description, try to find it nearby
        if len(parts) >= 2 and not re.search(r"[A-Za-z]", parts[1]):
            # Try backward merge
            if i > 0:
                prev_line = raw_lines[i - 1].strip()
                if len(re.split(r"\s{2,}", prev_line)) == 1:
                    parts.insert(1, prev_line)
                    skip_indices.add(i - 1)

            # Try forward merge
            elif i + 1 < len(raw_lines):
                next_line = raw_lines[i + 1].strip()
                if len(re.split(r"\s{2,}", next_line)) == 1:
                    parts.insert(1, next_line)
                    skip_indices.add(i + 1)

        # --- Extra fix: Split index + part number if mashed into one string
        if parts and re.match(r"^\d+\s+[A-Z0-9–\-]+$", parts[0]):
            split_part = re.split(r"\s+", parts[0], maxsplit=1)
            parts = split_part + parts[1:]

        structured_output.append(parts)

    # --- Final clean-up: remove leftover rows with only a single description cell
    structured_output = [
        row for row in structured_output
        if not (len(row) == 1 and re.match(r"^[A-Z ,\-0-9]+$", row[0]))
    ]

    return structured_output

def convert_to_table(pdf_id, year, brand, model, batch_id, structured_output):
    section = c_name = prev_section = prev_c_name = prev_ref_no = ""

    for line in structured_output:
        if line[0]:
            return

In [ ]:
# Everything works, only need to structure it into a table

# Format 1: Yamaha
pdf_1 = "Manuals/AEROX 155 '19 (B65P, B65R, B65S).pdf"
pdf_2= "Manuals/FJR1300A '15 (1MCH, 1MCG).PDF"
pdf_filepath = pdf_1

brand = "Yamaha"
year = extract_year(pdf_filepath, brand)
batch_id = extract_batch_id(pdf_filepath, brand)
model = extract_model(pdf_filepath)
pdf_id = model + '_' + batch_id

raw_lines = extract_raw_text(pdf_filepath)
structured_data = structure_raw_text(raw_lines)
master_parts_list = convert_to_table(pdf_id, year, brand, model, batch_id, structured_data)

#print(master_parts_list)
for row in structured_data:
    print(row)

['FIG.', '1', 'CYLINDER']
['REF.', 'CG', 'CH']
['NO.', 'PART NO.', 'DESCRIPTION', 'M', 'M', 'REMARKS']
['1', '1']
['1', '1MC–11102–00', 'CYLINDER HEAD ASSY', '1', '1']
['2', '5JW–11133–10', '.GUIDE, VALVE 1', '8', '8']
['3', '3P6–11134–10', '.GUIDE, VALVE 2', '8', '8']
['4', '93440–09141', '.CIRCLIP', '8', '8']
['5', '90340–27129', '.PLUG, STRAIGHT SCREW', '1', '1']
['6', '91808–16023', '.PIN, DOWEL', '16', '16']
['7', '90105–06026', '.BOLT, FLANGE', '20', '20']
['8', '95612–08620', 'BOLT, STUD', '8', '8']
['9', '94701–00415', 'PLUG, SPARK (NGK CPR8EA-9)', '4', '4']
['10', '90119–10004', 'BOLT, WITH WASHER', '10', '10']
['11', '95022–08012', 'BOLT, FLANGE', '1', '1']
['12', '90430–08119', 'GASKET', '1', '1']
['13', '1MC–11191–00', 'COVER, CYLINDER HEAD 1', '1', '1']
['14', '91810–20001', 'PIN, DOWEL', '4', '4']
['15', '1MC–11193–00', 'GASKET, HEAD COVER 1', '1', '1']
['16', '90109–066F0', 'BOLT', '8', '8']
['17', '5VY–1111G–00', 'RUBBER, MOUNT 1', '8', '8']
['18', '5JW–11181–10', 'GASK

In [ ]:
# Clean code:
def reconstruct_lines_from_chars(chars, y_tolerance=2.5):
    lines = defaultdict(list)

    for c in chars:
        # Use midpoint instead of top
        y_center = c["top"] + (c["height"] / 2)
        y_bucket = round(y_center / y_tolerance)
        lines[y_bucket].append(c)

    line_texts = []
    for y in sorted(lines.keys()):
        chars_in_line = sorted(lines[y], key=lambda c: c["x0"])
        line = ""
        prev_x = None

        for char in chars_in_line:
            x = char["x0"]
            text = char["text"]

            if prev_x is not None:
                gap = x - prev_x
                if gap > 1.5:
                    line += " " * int(gap / 2.5)

            line += text
            prev_x = char["x1"]

        line_texts.append((y, line.rstrip()))

    return line_texts

def extract_raw_text(pdf_path):
    output_lines = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            chars = page.chars
            raw_lines = reconstruct_lines_from_chars(chars, y_tolerance=3.0)

            # Skip non-parts pages
            if not raw_lines or not raw_lines[0][1].strip().startswith("FIG."):
                continue

            for _, line in raw_lines:
                stripped_line = line.strip()
                if stripped_line:
                    output_lines.append(stripped_line)

    return output_lines

def structure_raw_text(raw_lines):
    # ignore for now
    return

# Format 1: Yamaha
pdf_1 = "Manuals/AEROX 155 '19 (B65P, B65R, B65S).pdf"
pdf_2= "Manuals/FJR1300A '15 (1MCH, 1MCG).PDF"
# Format 2: Honda
pdf_3 = "Manuals/CRF1000 A_PC_13MJPG02_(G.H).pdf"
pdf_4 = "Manuals/NC750XAP_13MKWM02_PC_2022_2023.pdf"

raw_lines = extract_raw_text(pdf_1)